# Running Wflow using the ewatercycle system

### 1. Copy an example case
To run WFlow, we need a complete parameterset. The easiest way to obtain a valid model configuration is by copying it from https://github.com/openstreams/wflow/raw/master/examples/. We can use `ewatercycle.parametersetdb` to easily copy on of these example cases to a folder called `./wflow_example_case`.

In [ ]:
from ewatercycle.parametersetdb import build_from_urls

# Obtain an example case for testing the model
parameterset = build_from_urls(
    config_format='ini', config_url='https://github.com/openstreams/wflow/raw/master/examples/wflow_rhine_sbm_nc/wflow_sbm_NC.ini',
    datafiles_format='svn', datafiles_url='https://github.com/openstreams/wflow/trunk/examples/wflow_rhine_sbm_nc',
)
parameterset.save_datafiles('./wflow_example_case_nc')

### 2. Verify your configuration settings

If you are working on a curated environment like https://lab.ewatercycle.org/, the default configuration should cover 90% of all use cases. However, for the other 10% here is a brief explanation of the relevant configuration settings. 

The ewatercycle configuration file is typcially stored in `~/ewatercycle/ewatercycle.yaml` or `/etc/ewatercycle.yaml` and can be imported with `from ewatercycle import CFG`. Relevant settings for WFLow are:

```Yaml
container_engine: docker # or singularity
output_dir: './' # the directory where the model work_dir will be created
```

In order for this to work, you need to make sure that the actual docker or singularity images are available on the system. This can be done with:

- Docker: `docker pull ewatercycle/wflow-grpc4bmi:latest`
- Singularity: `singularity pull docker://ewatercycle/wflow-grpc4bmi:latest`

### 3. Setting up the model

In [1]:
import ewatercycle.models
ewatercycle.models.Wflow.available_versions

/home/peter/miniconda3/envs/ewatercycle/lib/python3.9/site-packages/esmvalcore/experimental/_warnings.py:18: UserWarning: 
  Thank you for trying out the new ESMValCore API.
  Note that this API is experimental and may be subject to change.
  More info: https://github.com/ESMValGroup/ESMValCore/issues/498
/home/peter/miniconda3/envs/ewatercycle/lib/python3.9/site-packages/esmvalcore/experimental/config/_config_validators.py:254: ESMValToolDeprecationWarning: `write_plots` will be removed in 2.4.0.
/home/peter/miniconda3/envs/ewatercycle/lib/python3.9/site-packages/esmvalcore/experimental/config/_config_validators.py:255: ESMValToolDeprecationWarning: `write_netcdf` will be removed in 2.4.0.


'2020.1.1'

In [2]:
# Load a parameterset
parameter_set = ewatercycle.models.wflow.WflowParameterSet(
    input_data = "./wflow_example_case_nc/",
    default_config = "./wflow_example_case_nc/wflow_sbm_NC.ini",
)
print(parameter_set)

Wflow parameterset
------------------
Directory: /home/peter/ewatercycle/ewatercycle/examples/wflow_example_case_nc
Default configuration file: /home/peter/ewatercycle/ewatercycle/examples/wflow_example_case_nc/wflow_sbm_NC.ini


In [3]:
# Load forcing data from an external source
import ewatercycle.forcing
forcing = ewatercycle.forcing.load_foreign(
    directory = "../",
    target_model = 'wflow',
    start_time = '1991-01-01T00:00:00Z',
    end_time = '1991-12-31T00:00:00Z',
    forcing_info = dict(
        # Additional information about the external forcing data needed for the model configuration
        netcdfinput = '../test_forcing_data.nc',
        Precipitation = "/P",
        EvapoTranspiration = "/PET",
        Temperature = "/TEMP"
    )
)
print(forcing)

Forcing data for Wflow
----------------------
Directory: ../
Start time: 1991-01-01T00:00:00Z
End time: 1991-12-31T00:00:00Z
Shapefile: None
Additional information for model config:
  - netcdfinput: ../test_forcing_data.nc
  - Precipitation: /P
  - Temperature: /TEMP
  - EvapoTranspiration: /PET
  - Inflow: None


In [4]:
# Create the model instance and inspect the pre-configured parameters
model = ewatercycle.models.Wflow(version='2020.1.1', parameter_set=parameter_set, forcing=forcing)
model.parameters

[('start_time', '1991-01-01T00:00:00Z'), ('end_time', '1991-12-31T00:00:00Z')]

In [5]:
# Set up the model; this is where you can potentially overwrite some of the parameters
cfg_file, cfg_dir = model.setup(end_time = "1991-02-28T00:00:00Z")

In [6]:
# See the run directory and model configuration file
print(cfg_file)
print(cfg_dir)

/home/peter/ewatercycle/ewatercycle/examples/wflow_20210611_102715/wflow_ewatercycle.ini
/home/peter/ewatercycle/ewatercycle/examples/wflow_20210611_102715


#### Update the config file

At this point, we have a preliminary configuration file. You can open it and modify. For the example below, you will have to add the variable "RiverRunoff" to the API section:

```
[API]
RiverRunoff = 2, m/s
```

After that, we can start up the model

### 4. Running the model

In [7]:
# Initialize the model
model.initialize(str(cfg_file))  # This conversion to string shouldn't be necessary

In [8]:
# Inspect the initial values for RiverRunoff
model.get_value("RiverRunoff")  # WFlow interally uses a fill-value for masked data of -999

array([-999., -999., -999., ..., -999., -999., -999.])

In [9]:
# You can also view get the data as xarray data-array
model.get_value_as_xarray("RiverRunoff")  # this automatically converts the -999 values to NaN

<xarray.DataArray 'RiverRunoff' (latitude: 169, longitude: 187)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * longitude  (longitude) float64 5.227 5.264 5.3 5.337 ... 11.97 12.01 12.05
  * latitude   (latitude) float64 45.89 45.93 45.97 46.0 ... 51.98 52.02 52.05
    time       object 1990-12-31 00:00:00
Attributes:
    units:     m/s

In [ ]:
# Run the model until complete
while model.time < model.end_time:
    print(model.time)
    model.update()

662601600.0


In [ ]:
# Get the final output and create a simple plot
da = model.get_value_as_xarray("RiverRunoff")
da.plot(robust=True, cmap='GnBu')

In [ ]:
# Cleaning up
model.finalize()
del(model)